In [12]:
import json
from transformers import T5ForConditionalGeneration, T5Tokenizer
from rank_bm25 import BM25Okapi
import torch

# Load T5 model and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Load menu data from the uploaded file
with open('mockMenu 1.json', 'r') as f:
    menu_data = json.load(f)['menu']

# Step 1: Prepare the corpus for BM25 by combining 'itemName' and 'description' fields
corpus = [
    (item['itemName'] + " " + (item['description'] if item['description'] else ""))
    for item in menu_data
]

# Initialize BM25 with the corpus
bm25 = BM25Okapi([doc.split() for doc in corpus])

### Step 2: Intent Recognition using T5 ###
def classify_intent_with_t5(query):
    input_text = f"classify intent: {query}"
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=20, num_beams=2)
    intent = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    return intent

### Step 3: Document Retrieval with BM25 ###
def retrieve_document(query):
    # Exact match: first check if the menu item name directly matches the query
    for item in menu_data:
        if item['itemName'].lower() in query.lower():
            return item
    
    # If no exact match, use BM25 for ranking
    tokenized_query = query.split()
    doc_scores = bm25.get_scores(tokenized_query)
    top_doc_index = torch.argmax(torch.tensor(doc_scores)).item()
    retrieved_item = menu_data[top_doc_index]
    return retrieved_item

### Step 4: Find Minimum Prep Time Dish ###
def find_min_prep_time_dish():
    min_item = min(menu_data, key=lambda x: int(x['prepTimeInMins']))
    return min_item

### Step 5: Generate Response based on Query ###
def generate_response_with_t5(query):
    if 'description' in query.lower():
        # Retrieve the correct item based on the query
        retrieved_item = retrieve_document(query)
        response = f"{retrieved_item['itemName']} is described as: {retrieved_item['description']}."
    elif 'price' in query.lower():
        retrieved_item = retrieve_document(query)
        response = f"The price of {retrieved_item['itemName']} is ${retrieved_item['price']}."
    elif 'allergy' in query.lower() or 'nuts' in query.lower():
        retrieved_item = retrieve_document(query)
        allergens = retrieved_item['allergicInfo'].upper()
        if 'NUTS' in allergens:
            response = f"Yes, {retrieved_item['itemName']} contains nuts."
        else:
            response = f"No, {retrieved_item['itemName']} does not contain nuts."
    elif 'least prep time' in query.lower() or 'min prep time' in query.lower():
        min_dish = find_min_prep_time_dish()
        response = f"The dish with the least prep time is {min_dish['itemName']} with a prep time of {min_dish['prepTimeInMins']} minutes."
    else:
        # Fallback to a general response using T5 generation
        retrieved_item = retrieve_document(query)
        input_text = f"generate response: {query} Context: {retrieved_item['description']}."
        inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
        outputs = model.generate(inputs, max_length=50, num_beams=5, early_stopping=True)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


# Question 1

In [15]:

### Main Flow ###
input_json = '''
{
    "query": "Drink something hot"
}
'''

# Parse input JSON
input_data = json.loads(input_json)
query = input_data['query']

# Step 1: Classify the intent using T5
intent = classify_intent_with_t5(query)

# Step 2: Generate the response based on the query
response = generate_response_with_t5(query)

# Output the result in JSON format
output_json = {
    'intent': intent,
    "response": response
}

# Print the output in JSON format
output = json.dumps(output_json, indent=4)
print(output)

{
    "intent": "Drink something hot",
    "response": "Drink something hot Context: Drink something hot Context: A healthy breakfast option made with semolina and vegetables lightly tempered with spices cooked to perfection."
}


# Question 2

In [14]:

### Main Flow ###
input_json = '''
{
    "query": "Do you offer any child friendly dishes"
}
'''

# Parse input JSON
input_data = json.loads(input_json)
query = input_data['query']

# Step 1: Classify the intent using T5
intent = classify_intent_with_t5(query)

# Step 2: Generate the response based on the query
response = generate_response_with_t5(query)

# Output the result in JSON format
output_json = {
    'intent': intent,
    "response": response
}

# Print the output in JSON format
output = json.dumps(output_json, indent=4)
print(output)

{
    "intent": "Poss",
    "response": "Context: A healthy breakfast option made with semolina and vegetables lightly tempered with spices cooked to perfection Context: A healthy breakfast option made with semolina and vegetables lightly tempered with spices cooked to perfection."
}


# Question 3

In [16]:

### Main Flow ###
input_json = '''
{
    "query": "Recommend something for fever"
}
'''

# Parse input JSON
input_data = json.loads(input_json)
query = input_data['query']

# Step 1: Classify the intent using T5
intent = classify_intent_with_t5(query)

# Step 2: Generate the response based on the query
response = generate_response_with_t5(query)

# Output the result in JSON format
output_json = {
    'intent': intent,
    "response": response
}

# Print the output in JSON format
output = json.dumps(output_json, indent=4)
print(output)

{
    "intent": "classify intent: Recommend something for fever",
    "response": "Recommend something for fever Context: Extra side for poori, dosa."
}


In [17]:

### Main Flow ###
input_json = '''
{
    "query": "Vegan dishes"
}
'''

# Parse input JSON
input_data = json.loads(input_json)
query = input_data['query']

# Step 1: Classify the intent using T5
intent = classify_intent_with_t5(query)

# Step 2: Generate the response based on the query
response = generate_response_with_t5(query)

# Output the result in JSON format
output_json = {
    'intent': intent,
    "response": response
}

# Print the output in JSON format
output = json.dumps(output_json, indent=4)
print(output)

{
    "intent": "Vegan dishes",
    "response": "Vegan dishes Context: A healthy breakfast option made with semolina and vegetables lightly tempered with spices cooked to perfection."
}
